In [1]:
!pip3 install pyspark

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from pyspark import SparkContext

In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder \
    .master("local") \
    .appName("Test it 2") \
    .getOrCreate()
sc = spark.sparkContext


In [4]:
spark

### The filter transformation

In [5]:
raw_data = sc.textFile("konstytucja.txt")

In [6]:
raw_data.count()

808

In [7]:
sentences = raw_data.filter(lambda x: "" != x)

In [8]:
sentences.count()

236

### Map transformation

In [10]:
words_batch = sentences.map(lambda x: x.split(" "))

In [11]:
words_batch.take(10)

[['Konstytucja', 'Rzeczypospolitej', 'Polskiej/Rozdział', '1'],
 ['Pobrano', 'z', 'Wikiźródeł', 'dnia', '02/14/20'],
 ['Konstytucja', 'Rzeczypospolitej', 'Polskiej'],
 ['Rozdział', '1'],
 ['Konstytucja', 'Rzeczypospolitej', 'Polskiej'],
 ['Rozdział',
  '1',
  'Rzeczpospolita',
  '—',
  'Rozdział',
  '2',
  'Wolności,',
  'prawa',
  'i',
  'obowiązki',
  'człowieka',
  'i',
  'obywatela,',
  'zasady',
  'ogólne',
  'wolności,',
  'prawa',
  'osobiste,',
  'wolności',
  'i',
  'prawa',
  'polityczne,',
  'wolności',
  'i',
  'prawa',
  'ekonomiczne,',
  'socjalne',
  'i',
  'kulturalne,',
  'środki',
  'ochrony',
  'wolności',
  'i',
  'praw,',
  'obowiązki',
  '—',
  'Rozdział',
  '3',
  'Źródła',
  'prawa',
  '—',
  'Rozdział',
  '4',
  'Sejm',
  'i',
  'Senat,',
  'wybory',
  'i',
  'kadencja,',
  'posłowie',
  'i',
  'senatorowie,',
  'organizacja',
  'i',
  'działanie,',
  'referendum',
  '—',
  'Rozdział',
  '5',
  'Prezydent',
  'Rzeczypospolitej',
  'Polskiej',
  '—',
  'Rozdział

### flatMap transformation

In [14]:
words = sentences.flatMap(lambda x: x.split(" "))
words = words.map(lambda x: x.lower())

In [15]:
words_occurence = words.map(lambda word: (word, 1))

In [16]:
wordCounts = words_occurence.reduceByKey(lambda a,b: a + b)

In [17]:
wordCounts.map(lambda x: (x[1], x[0])) \
    .sortByKey(ascending=False) \
    .take(5)

[(174, 'i'), (151, 'w'), (92, 'art.'), (75, 'rozdział'), (60, '—')]

In [18]:
!wget https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt

--2020-02-14 18:34:02--  https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt
Resolving raw.githubusercontent.com... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2016 (2.0K) [text/plain]
Saving to: 'polish.stopwords.txt'

polish.stopwords.tx 100%[===================>]   1.97K  --.-KB/s    in 0s      

2020-02-14 18:34:03 (26.2 MB/s) - 'polish.stopwords.txt' saved [2016/2016]



In [19]:
stop_words = sc.textFile("polish.stopwords.txt").collect()

In [20]:
stop_words

['a',
 'aby',
 'ach',
 'acz',
 'aczkolwiek',
 'aj',
 'albo',
 'ale',
 'alez',
 'ależ',
 'ani',
 'az',
 'aż',
 'bardziej',
 'bardzo',
 'beda',
 'bedzie',
 'bez',
 'deda',
 'będą',
 'bede',
 'będę',
 'będzie',
 'bo',
 'bowiem',
 'by',
 'byc',
 'być',
 'byl',
 'byla',
 'byli',
 'bylo',
 'byly',
 'był',
 'była',
 'było',
 'były',
 'bynajmniej',
 'cala',
 'cali',
 'caly',
 'cała',
 'cały',
 'ci',
 'cie',
 'ciebie',
 'cię',
 'co',
 'cokolwiek',
 'cos',
 'coś',
 'czasami',
 'czasem',
 'czemu',
 'czy',
 'czyli',
 'daleko',
 'dla',
 'dlaczego',
 'dlatego',
 'do',
 'dobrze',
 'dokad',
 'dokąd',
 'dosc',
 'dość',
 'duzo',
 'dużo',
 'dwa',
 'dwaj',
 'dwie',
 'dwoje',
 'dzis',
 'dzisiaj',
 'dziś',
 'gdy',
 'gdyby',
 'gdyz',
 'gdyż',
 'gdzie',
 'gdziekolwiek',
 'gdzies',
 'gdzieś',
 'go',
 'i',
 'ich',
 'ile',
 'im',
 'inna',
 'inne',
 'inny',
 'innych',
 'iz',
 'iż',
 'ja',
 'jak',
 'jakas',
 'jakaś',
 'jakby',
 'jaki',
 'jakichs',
 'jakichś',
 'jakie',
 'jakis',
 'jakiś',
 'jakiz',
 'jakiż',
 'jak

In [49]:
counter = wordCounts.map(lambda x: (x[1], x[0])) \
    .filter(lambda x: x[1] not in stop_words) \
    .sortByKey(ascending=False)

#### Creating a RDD from a file
The most common way of creating an RDD is to load it from a file. Notice that Spark's textFile can handle compressed files directly.

In [50]:
counter.take(15)

[(92, 'art.'),
 (75, 'rozdział'),
 (60, '—'),
 (50, 'rzeczypospolitej'),
 (33, 'prawa'),
 (33, 'ustawy'),
 (31, 'polskiej'),
 (30, 'konstytucji'),
 (27, 'wolności'),
 (25, 'życie'),
 (24, 'dnia'),
 (23, 'rada'),
 (23, 'ministrów'),
 (19, 'wejścia'),
 (18, 'rzeczpospolita')]

### The collect action

Basically it will get all the elements in the RDD into memory for us to work with them. For this reason it has to be used with care, specially when working with large RDDs.

In [51]:
counted_words = counter.collect()

In [53]:
counted_words[:20]

[(92, 'art.'),
 (75, 'rozdział'),
 (60, '—'),
 (50, 'rzeczypospolitej'),
 (33, 'prawa'),
 (33, 'ustawy'),
 (31, 'polskiej'),
 (30, 'konstytucji'),
 (27, 'wolności'),
 (25, 'życie'),
 (24, 'dnia'),
 (23, 'rada'),
 (23, 'ministrów'),
 (19, 'wejścia'),
 (18, 'rzeczpospolita'),
 (18, 'sejm'),
 (16, 'ust.'),
 (15, 'konstytucja'),
 (15, '1'),
 (14, 'prezydent')]